In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/

 2025_ChemE498-599_Syllabus.pdf
'2025_CHEME 498_Syllabus_final_upload (1).pdf'
'2025_CHEME 498_Syllabus_final_upload.pdf'
'310 creativity script.gdoc'
'326 cheat sheet.gslides'
 485_HW2.csv
'Capstone PGST'
'CHEM E 375 Spring 2024 Syllabus (1).pdf'
'CHEM E 375 Spring 2024 Syllabus (2).pdf'
'CHEM E 375 Spring 2024 Syllabus (3).pdf'
'CHEM E 375 Spring 2024 Syllabus (4).pdf'
'CHEM E 375 Spring 2024 Syllabus.pdf'
'CHEM E 436 TEAM 3'
'CHEM E 498 Final Project Presentation.gslides'
'Colab Notebooks'
'Copy of E-FIG Compass Keywords (1).gdoc'
'Copy of E-FIG Compass Keywords.gdoc'
 Deines.pdf
'INFO 200 Final Lab Review.gslides'
'NME 220 HW1.gdoc'
'NME 220 HW 3.gdoc'
'Physics 123AN5'
 Quiz+3+Solutions.docx
'resume!!.pdf'
 SA2.gdoc
 Synthesis.gdoc
'Works Cited_5e2a1c5a-117b-4ef5-b7d4-9930a91a24ab.gdoc'


In [2]:
# load data
df = pd.read_excel("Port Gamble Ferrell Gas Consumption.xlsx", header=3)

# remove fully empty columns
df = df.dropna(axis=1, how='all')

# rename columns
df.columns = df.columns.str.strip()
df = df.dropna(axis=1, how='all').rename(columns={
    'Transaction date': 'date',
    'Transaction Description': 'description',
    'Doc Number': 'doc_number',
    'Sale Amount': 'sale_amount',
    'Gallons': 'gallons',
    'Document Reference': 'reference',
    'Payment Amount': 'payment_amount',
    'Group': 'group_id',
    'Gallon Consumption': 'gallon_consumption',
    'Running total': 'running_total'
})

# drop rows with no transaction date (non-data rows)
df = df[df['date'].notna()]

# convert data types
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['gallons'] = pd.to_numeric(df['gallons'], errors='coerce').fillna(0)
df['sale_amount'] = pd.to_numeric(df['sale_amount'], errors='coerce')
df['payment_amount'] = pd.to_numeric(df['payment_amount'], errors='coerce')
df['running_total'] = pd.to_numeric(df['running_total'], errors='coerce')

# remove negative gallons and corresponding positive entries
dates_remove_all = pd.to_datetime(["2010-12-13","2010-12-27","2022-04-14"])
dates_remove_neg = pd.to_datetime(["2010-12-30", "2020-02-18", "2022-04-15"])
df = df[~(df['date'].isin(dates_remove_all) | (df['date'].isin(dates_remove_neg) & (df['gallons'] < 0)))]

# sort chronologically
df = df.sort_values('date').reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Port Gamble Ferrell Gas Consumption.xlsx'

In [ ]:
# focus only on actual deliveries
deliveries = df[df['gallons'] > 0]

# define volume bins (gallons)
bins = np.linspace(0, 1750, 36)

# count frequency in each bin
counts = pd.cut(deliveries['gallons'], bins=bins, right=False).value_counts().sort_index()

# use left bin edges for x positions
x = bins[:-1]
y = counts.values

plt.figure(figsize=(14, 6))
plt.bar(x, y, width=np.diff(bins), edgecolor='black', align='edge')
plt.xticks(bins[::1], rotation=45, ha='right')
plt.xlim(0, 1750)
plt.ylim(0, 40)
plt.xlabel("Delivery Volume (gallons)")
plt.ylabel("Number of Deliveries")
plt.title("Frequency of Propane Delivery Volumes")
plt.tight_layout()
plt.show()

In [ ]:
# monthly total gallons
monthly = (df.set_index('date').resample('ME')['gallons'].sum())
plt.plot(monthly.index, monthly.values)
plt.xlim(df.loc[df['gallons'] > 0, 'date'].min() - pd.Timedelta(days=21), df.loc[df['gallons'] > 0, 'date'].max() + pd.Timedelta(days=7))
plt.ylim(0,7000)
plt.xlabel("Date")
plt.ylabel("Gallons Consumed")
plt.title("Monthly Propane Consumption Over Time")
plt.show()

In [ ]:
# rolling average gallons
rolling_12 = monthly.rolling(window=12, min_periods=6).mean()
plt.plot(monthly.index, monthly.values, alpha=0.4, label="Monthly Consumption")
plt.plot(rolling_12.index, rolling_12.values, linewidth=2, label="12-Month Rolling Average")
plt.xlim(df.loc[df['gallons'] > 0, 'date'].min() - pd.Timedelta(days=21), df.loc[df['gallons'] > 0, 'date'].max() + pd.Timedelta(days=7))
plt.ylim(0,7000)
plt.xlabel("Date")
plt.ylabel("Gallons Consumed")
plt.title("Propane Consumption with Rolling Average")
plt.legend()
plt.show()

In [ ]:
# annual total gallons
annual = (df.assign(year=df['date'].dt.year).groupby('year')['gallons'].sum())
plt.bar(annual.index, annual.values)
plt.xlim(2004,2026)
plt.ylim(0,35000)
plt.xticks([2005, 2010, 2015, 2020, 2025])
plt.xlabel("Year")
plt.ylabel("Total Gallons Consumed")
plt.title("Annual Propane Consumption")
plt.show()

In [ ]:
# df_recent --> only data from 2016-2025
df_recent = df[(df['date'].dt.year >= 2016) & (df['date'].dt.year <= 2025)]

# monthly average gallons by calendar month
monthly_avg = (df_recent.assign(month=df['date'].dt.month).groupby('month')['gallons'].mean())
plt.plot(monthly_avg.index, monthly_avg.values, marker='o')
plt.xlim(0.8,12.2)
plt.ylim(0, 600)
plt.xticks(ticks=range(1, 13),labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.xlabel("Month")
plt.ylabel("Average Gallons Consumed")
plt.title("Average Propane Consumption by Month, 2016-2025")
plt.show()

In [ ]:
# propane consumption heatmap
df_2005 = df[df['date'] >= '2005-01-01']
monthly_totals = (df_2005.assign(year=df['date'].dt.year,month=df_2005['date'].dt.month).groupby(['year', 'month'])['gallons'].sum().unstack())
plt.imshow(monthly_totals, aspect='auto', cmap='YlOrRd', vmin=0, vmax=5000)
plt.colorbar(label="Gallons Consumed")
plt.xlabel("Month")
plt.ylabel("Year")
plt.title("Monthly Propane Consumption Heatmap")
plt.xticks(ticks=range(12),labels=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
plt.yticks(ticks=range(len(monthly_totals.index)),labels=monthly_totals.index)
plt.show()

In [ ]:
# price per gallon over time
df_price = df[(df['gallons'] > 0) & (df['description'].str.contains('PROPANE', case=False, na=False))].copy()
monthly_totals = (df_price.set_index('date').resample('ME').agg(dollars=('sale_amount', 'sum'),gallons=('gallons', 'sum')))
monthly_price = monthly_totals['dollars'] / monthly_totals['gallons']
plt.plot(monthly_price.index, monthly_price.values)
plt.xlim(df.loc[df['gallons'] > 0, 'date'].min() - pd.Timedelta(days=21), df.loc[df['gallons'] > 0, 'date'].max() + pd.Timedelta(days=7))
plt.ylim(0,4)
plt.xlabel("Date")
plt.ylabel("Price per Gallon ($)")
plt.title("Average Propane Price per Gallon Over Time")
plt.show()

In [ ]:
# annual consumption and spend
annual_summary = (df[(df['date'] >= '2005-01-01') & (df['date'] <= '2025-12-31')].assign(year=df['date'].dt.year).groupby('year').agg(total_gallons=('gallons', 'sum'),total_spend=('sale_amount', 'sum')))
fig, ax1 = plt.subplots()
line1, = ax1.plot(annual_summary.index, annual_summary['total_gallons'], linestyle='-', label='Total Gallons')
ax1.set_xticks([2005, 2010, 2015, 2020, 2025])
ax1.set_xlim(2004.8, 2025.2)
ax1.set_ylim(0, 35000)
ax1.set_xlabel("Year")
ax1.set_ylabel("Total Gallons Consumed")
ax2 = ax1.twinx()
line2, = ax2.plot(annual_summary.index, annual_summary['total_spend'], linestyle='--', color = 'red', label='Total Spend ($)')
ax2.set_ylim(0, 60000)
ax2.set_ylabel("Total Annual Propane Spend ($)")
ax1.legend([line1, line2], [l.get_label() for l in [line1, line2]], loc='upper left')
plt.title("Annual Propane Consumption and Annual Spend")
plt.show()

In [ ]:
# annual propane usage over past three years (for ReOPT heating estimates)
end_year = df['date'].dt.year.max() - 1
start_year = end_year - 2

df_3yr = df[(df['date'].dt.year >= start_year) & (df['date'].dt.year <= end_year)].copy()

fuel = df_3yr[df_3yr['gallons'] > 0].copy()

annual_gallons = (fuel.assign(year=fuel['date'].dt.year).groupby('year')['gallons'].sum())

# gallons
print("\nAnnual propane consumption (gallons):")
print("------------------------------------")
print(annual_gallons.rename("Total Gallons").to_string())

avg_gallons = round(annual_gallons.mean(), 1)
print(f"\nAverage annual propane consumption over {start_year}–{end_year}: "f"{avg_gallons} gallons")

# convert to kWh
KWH_PER_GALLON = 91500 / 3412  # ≈ 26.8

annual_heating_kwh = annual_gallons * KWH_PER_GALLON

print("\nEstimated annual heating energy input (kWh, thermal, fuel basis):")
print("------------------------------------------------")
print(annual_heating_kwh.rename("Heating Energy (kWh)").to_string())

avg_kwh = round(annual_heating_kwh.mean(), 1)
print(f"\nAverage annual heating energy input over {start_year}–{end_year}: "f"{avg_kwh} kWh (thermal, fuel basis)")

# incorporate efficiency
BOILER_EFFICIENCY = 0.80

annual_heating_kwh_useful = annual_gallons * KWH_PER_GALLON * BOILER_EFFICIENCY

print("\nEstimated useful heating demand (kWh, thermal, assuming 80% efficiency):")
print("------------------------------------------------")
print(annual_heating_kwh_useful.rename("Heating Energy (kWh)").to_string())

avg_kwh = round(annual_heating_kwh_useful.mean(), 1)
print(f"\nAverage estimated useful heating demand over {start_year}–{end_year}: "f"{avg_kwh} kWh (thermal, assuming 80% efficiency)")

In [ ]:
# propane usage for 2024 only
fuel_2024 = df[(df['date'].dt.year == 2024) & (df['gallons'] > 0)].copy()

total_gallons_2024 = fuel_2024['gallons'].sum()

print(f"Total propane consumption in 2024: {total_gallons_2024:.1f} gallons")

KWH_PER_GALLON = 91500 / 3412  # ≈ 26.8 kWh/gal

heating_kwh_2024 = total_gallons_2024 * KWH_PER_GALLON

print(f"2024 heating energy input: {heating_kwh_2024:.1f} kWh (thermal, fuel basis)")

BOILER_EFFICIENCY = 0.80

useful_heating_kwh_2024 = heating_kwh_2024 * BOILER_EFFICIENCY

print(
    f"2024 useful heating demand: "
    f"{useful_heating_kwh_2024:.1f} kWh (thermal, 80% efficiency)"
)

In [ ]:
# ------------------------------------------------------------
# Monthly propane totals
# ------------------------------------------------------------

monthly_propane = (
    df_recent.assign(year=df_recent['date'].dt.year,
                     month=df_recent['date'].dt.month)
             .groupby(['year','month'])['gallons']
             .sum()
             .reset_index()
)

# ------------------------------------------------------------
# Load monthly temperature data
# ------------------------------------------------------------

temps = pd.read_csv("PGST Monthly average temperatures.csv")
temps['Date'] = pd.to_datetime(temps['Date'])

temps['year'] = temps['Date'].dt.year
temps['month'] = temps['Date'].dt.month

temps = temps[['year','month','Temperature_F']]

# ------------------------------------------------------------
# Merge by year/month
# ------------------------------------------------------------

reg_df = monthly_propane.merge(temps, on=['year','month'], how='inner')

reg_df = reg_df.rename(columns={'Temperature_F':'temp_F'})


# ------------------------------------------------------------
# Linear regression
# ------------------------------------------------------------

x = reg_df['temp_F'].values
y = reg_df['gallons'].values

m, b = np.polyfit(x, y, 1)
x_line = np.linspace(x.min(), x.max(), 100)
y_line = m * x_line + b

r2 = np.corrcoef(x, y)[0, 1]**2


# ------------------------------------------------------------
# Plot
# ------------------------------------------------------------

plt.figure(figsize=(7, 6))

plt.scatter(x, y)
plt.plot(x_line, y_line)

plt.xlabel("Average Monthly Temperature (°F)")
plt.ylabel("Monthly Propane Consumption (gallons)")
plt.title(f"Propane vs Temperature\nSlope={m:.1f} gal/°F   R²={r2:.2f}")

plt.tight_layout()
plt.show()
